### generate pool data table

In [25]:
import cmocean as cmo
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import canyon_tools.readout_tools as rout
import canyon_tools.savitzky_golay as sg
import xarray as xr
import matplotlib.gridspec as gspec

In [26]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [27]:
sns.set_context('notebook')
sns.set_style('whitegrid')

In [28]:
sb_Ast = 29 # shelf break z-index Astoria
sb_Bar = 39 # shelf break z-index Barkley

ptr_fileB = '/data/kramosmu/results/TracerExperiments/UPW_10TR_BF4_BAR/01_Bar03/ptracersGlob.nc'
ptr_fileA = '/data/kramosmu/results/TracerExperiments/UPW_10TR_BF2_AST/01_Ast03/ptracersGlob.nc'


#### Get initial concentration at shelf break

In [29]:
tracers = ['Tr01','Tr02','Tr03','Tr04','Tr05','Tr06','Tr07','Tr08','Tr09','Tr10']
labels = ['Linear 01','Salinty 02','Oxygen 03','Nitrate 04','Silicate 05','Phosphate 06','Nitrous Oxide 07','Methane 08',
         'DIC 09', 'Alk 10']
colours = ['#332288','#44AA99','#117733','#999933','#DDCC77','#CC6677','#882255','#AA4499', 'brown', 'tan']
sb_conc_A = np.empty(len(labels))
sb_conc_B = np.empty(len(labels))

for ii, trac in zip(range(len(tracers)),tracers):
    for pfile,sb_array, sb_ind in zip([ptr_fileB, ptr_fileA],[sb_conc_A, sb_conc_B], [sb_Ast, sb_Bar]):
        with Dataset(pfile, 'r') as nbl:
            tr_profile = nbl.variables[trac][0,:,10,180]
            sb_array[ii] = tr_profile[sb_ind]

    

### Table all runs

In [30]:
labels_exp = ['Astoria', 'Ast. ARGO', 'Barkley', 'Bar. Path.']
labels_tra = ['Lin','Sal','Oxy','Nit','Sil','Pho','NiO','Met','DIC','Alk']
units = ['(muM)m3/s','PSU','(umol/kg)m3/s','(muM)m3/s','(muM)m3/s','muM)m3/s','(nM)m3/s','(nM)m3/s','(umol/kg)m3/s','(umol/kg)m3/s']

factors = [1,1,1,1,1,1,1E-3,1E-3,1,1]

print ("\t".join(['Run &', 
                  '$A_{pool}/A_{can}$ day 9 &',
                  'max $A_{pool}/A_{can}$) &',
                  '$C$ at day 9 &',
                  'max C &'
                  'max $\Delta C$ (%) &',
                  'mean adv BAC &'
                  'mean adv BAC anom skl'
                 ]))

exp_files = ['saved_calcs/pool_AST.nc',
             'saved_calcs/pool_ARGO.nc',
             'saved_calcs/pool_BAR.nc',
             'saved_calcs/pool_PATH.nc']

exp_files_BAC = ['saved_calcs/BAC_AST.nc',
                 'saved_calcs/BAC_ARGO.nc',
                 'saved_calcs/BAC_BAR.nc',
                 'saved_calcs/BAC_PATH.nc']


can_Area = [1.8E8,1.8E8,8.7E7,8.7E7]
sb_conc = [sb_conc_A, sb_conc_A, sb_conc_B, sb_conc_B]

for tr, unit, tr_lab, factor, ii, trac in zip(tracers, units, labels_tra, factors, range(len(units)), tracers):
    for file,file_bac, lab_exp, can_area, sb_c in zip(exp_files, exp_files_BAC, labels_exp, can_Area, sb_conc):
        with Dataset(file, 'r') as nbl:
            area = nbl.variables['area']
            area_can = nbl.variables['area_can']
            conc = nbl.variables['conc']
            A_day_9 = area[ii,-1]/can_area
            A_max = np.nanmax(area[ii,:])/can_area

            Conc = conc[ii,:]*factor
            C_sb = sb_c[ii]*factor
            C_day_9 = Conc[-1]
            if ii == 2: #oxygen
                C_max = np.nanmin(Conc[:])
            else:
                C_max = np.nanmax(Conc[:])
            
        
        with Dataset(file_bac,'r') as nbl:
            bac = nbl.variables['bac']
            bac_noc = nbl.variables['bac_noc']
            bac_adv = np.nanmean(bac[ii,8:]*factor)
            bac_anom = np.nanmean((bac[ii,8:]-bac_noc[ii,8:])/bac_noc[ii,8:])*100
            
        max_DC = ((C_max-C_sb)/C_sb)*100
        print ("\t".join(['%s %s &' %(lab_exp,tr_lab), 
                 '%1.2f &' %A_day_9,
                 '%1.2f &' %A_max,
                 '%1.2f &' %C_day_9,
                 '%1.2f &' %C_max, 
                 '%1.2f &' %max_DC, 
                 '%1.2f &' %bac_adv, 
                 '%1.2f skl' %bac_anom, 
                          
                ]))

Run &	$A_{pool}/A_{can}$ day 9 &	max $A_{pool}/A_{can}$) &	$C$ at day 9 &	max C &max $\Delta C$ (%) &	mean adv BAC &mean adv BAC anom skl
Astoria Lin &	38.91 &	38.91 &	8.31 &	9.25 &	28.12 &	6.95 &	18.50 skl
Ast. ARGO Lin &	22.80 &	22.80 &	8.32 &	8.71 &	20.72 &	6.73 &	14.82 skl
Barkley Lin &	6.60 &	11.18 &	9.84 &	9.95 &	10.32 &	6.16 &	0.06 skl
Bar. Path. Lin &	23.48 &	23.48 &	10.08 &	10.41 &	15.40 &	6.05 &	0.49 skl
Astoria Sal &	3.84 &	10.67 &	33.93 &	33.94 &	0.13 &	33.68 &	-0.00 skl
Ast. ARGO Sal &	2.66 &	7.11 &	33.94 &	33.94 &	0.12 &	33.68 &	0.03 skl
Barkley Sal &	3.44 &	7.83 &	33.95 &	33.95 &	0.04 &	33.51 &	-0.04 skl
Bar. Path. Sal &	7.88 &	15.04 &	33.96 &	33.96 &	0.06 &	33.46 &	-0.01 skl
Astoria Oxy &	35.26 &	35.26 &	96.20 &	86.66 &	-16.73 &	117.35 &	-5.93 skl
Ast. ARGO Oxy &	20.45 &	20.45 &	95.46 &	90.81 &	-12.74 &	118.74 &	-5.34 skl
Barkley Oxy &	208.20 &	210.12 &	123.30 &	123.30 &	45.49 &	131.72 &	0.56 skl
Bar. Path. Oxy &	204.45 &	210.12 &	127.97 &	127.97 &	50.99 &	134.96 &	-0.0

In [31]:
15.7*4

62.8

In [32]:
21.8*2

43.6

In [33]:
13*4


52